# МДЗ-2

## Золотарев Антон Олегович

# Ссылочки на участки кода

Задание 1. [ссылка на результат](https://storage.yandexcloud.net/bucket-zolotarev-bst/mhw2_task1.txt)

Задание 2 не сделал, ибо тупил с разворачиванием ноутбука как в семинаре 3

*Note: Выполнение задания 1 было осуществлено в консоли*

# Предварительные операции

Зайдём на мастер ноду и откроем датасет из прошлой МДЗ

```bash
eval `ssh-agent -s`
ssh-add lsml_ssh
ssh yc-user@62.84.125.253
eval `ssh-agent -s`
logout
ssh -A -J yc-user@62.84.125.253 ubuntu@rc1a-dataproc-c-0430dmrwee1whrn9.mdb.yandexcloud.net
cd ~/mhw1/7z-mhw1
head -n 5 VisitsStream.tsv
```

# Задание 1 (5 баллов)

В `VisitsStream.tsv` лежит информация про пользователей, которые открывают сайт. Используя классический Hadoop MapReduce необходимо посчитать топ 10 пользователей с самыми длинными по времени сессиями и время этой самой длинной сессии (в секундах).

Сессия определяется следующим образом - это окно времени, внутри которого временное расстояние от двух соседних посещений не более **15 минут**. 

Иными словами - если пользователь зашел на сайт в момент X и последнее предыдущее посещение сайта в момент Y было не позднее чем 15 минут назад, то сессия "продлевается" до текущего момента. Если же временное расстояние от X до Y более 15 минут, то считается, что предыдущая сессия закончилась в момент Y, а новая сессия началась в момент X.

Сессия может длится 0 секунд, если пользователь сделал всего 1 запрос в течение 30 минутного окна (в середине этого окна). Считается, что в начале у пользователя нет открытой сессии и что сессия автоматически заканчивается, когда записей больше не осталось.

Выводить нужно только уникальных пользователей и для каждого такого пользователя находить время самой длинной его сессии. 

При решении можно использовать произвольное количество MapReduce задач, но чем меньше, тем лучше. За излишне неоптимальное решение можно потерять балл. 

Полученный файл с топ 10 нужно будет выложить в облако, обеспечить публичный доступ до него и приложить к решению.

В ноутбуке должны присутствовать ячейки с 

1) Всеми необходимыми скриптами для работы ваших MapReduce задач

2) Командами запуска самих MapReduce задач

3) Ссылкой на итоговый результат работы в вашем облаке. Ссылки должны быть рабочими до того момента, как вашу домашку не проверят.


Посмотрим как дела в HDFS

```bash
hdfs dfs -ls /
```

Обрежем заголовки, и положим датасет в HDFS для возможности использования Hadoop MapReduce, после чего балансируем полученное

```bash
tail -n +2 VisitsStream.tsv > VisitsStreamRaw.tsv
hdfs dfs -mkdir -p /user/avito/data
hdfs dfs -put VisitsStreamRaw.tsv /user/avito/data/
sudo hdfs balancer
```

Чтобы решить задачу, необходимо декомпозировать поставленную задачу на следующие проблемы:
1. На мап-стадии объединяем UserID & ViewDate
2. Партишонер раскидывает строчки на редьюсер по ключу UserID и сортирует сначала по этому ключу, затем по дате
3. Редьюсер считает для каждого пользователя протяжённость самой длинной сессии
4. После этого получаем неотсортированный рейтинг пользователей с самыми длинными сессиями, переходим ко второй Map-Reduce операции
5. Маппим по составному ключу, состоящему из UserID & протяжённости его самой длинной сессии
6. Партишонер сортирует по самой длинной сессии
7. Единый редьюсер выводит лишь 10 самых крутых пользователей

In [4]:
%%writefile './top10_longest_sessions.py'

import collections
import sys
import csv
import datetime
from itertools import groupby, islice

def mapping():
    for row in csv.reader(iter(sys.stdin.readline, ''), delimiter='\t'):
        user_id, session_start = row[0], row[3]
        print("{}+{}\t".format(user_id.strip(), session_start.strip()))

def reducing():
    for user_id, session_starts in groupby(kv_stream('+'), lambda x: x[0]):
        user_session_starts = (x.strip() for _, x in session_starts)

        max_session_length = 0
        session_length = 0
        session = datetime.datetime.strptime(next(user_session_starts), '%Y-%m-%d %H:%M:%S.%f')
        for next_session in user_session_starts:
            next_session = datetime.datetime.strptime(next_session, '%Y-%m-%d %H:%M:%S.%f')
            curr_session_length = (next_session - session).total_seconds()
            if curr_session_length <= 900.0:
                session_length += curr_session_length
                if session_length > max_session_length:
                    max_session_length = session_length
            else:
                session_length = 0         
            session = next_session        
        print('{}\t{}'.format(user_id, max_session_length))
    
def top_10_mapping():
    for k, v in map(lambda x: x.split('\t'), sys.stdin):
        print("{}+{}\t".format(k.strip(), v.strip()))

def top_10_reducing():
    first_10_stream = islice(map(lambda x: x.split('+'), sys.stdin), 10)
    for user, session_len in first_10_stream:
        print("{}\t{}".format(user, session_len.strip()))
    collections.deque(sys.stdin, maxlen=0)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {'map': mapping,
    'reduce': reducing,
    'top_10_map': top_10_mapping,
    'top_10_reduce': top_10_reducing,
    }[mr_command]()

Writing ./top10_longest_sessions.py


После этого я закинул код в бакет и перенёс в отдельную папку для скриптов

```bash
hdfs dfs -get s3a://bucket-zolotarev-bst/top10_longest_sessions.py ~/scripts
```

Запускаем мап-редьюс по аналогии с семинарами

```bash
yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="length-max-session" \
-D mapreduce.job.reduces=3 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k1,1 -k2,2" \
-D mapreduce.map.output.key.field.separator='+' \
-D mapred.partitioner.class=org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-D mapreduce.partition.keypartitioner.options="-k1,1" \
-files ~/scripts/top10_longest_sessions.py \
-mapper "python3 top10_longest_sessions.py map" \
-reducer "python3 top10_longest_sessions.py reduce" \
-input /user/avito/data/ \
-output /user/avito/longest-sessions/
```

Выцепляем топ-10 пользователей с самыми длинными сессиями
```bash
yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top10-sessions" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files ~/scripts/top10_longest_sessions.py \
-mapper "python3 top10_longest_sessions.py top_10_map" \
-reducer "python3 top10_longest_sessions.py top_10_reduce" \
-input /user/avito/longest-sessions/ \
-output /user/avito/top10-longest-sessions/
```

```bash
hdfs dfs -cat /user/avito/top10-longest-sessions/part*
```

Файлик с полученным результатом - https://storage.yandexcloud.net/bucket-zolotarev-bst/mhw2_task1.txt
```bash
hdfs dfs -cp /user/avito/top10-longest-sessions/part* s3a://bucket-zolotarev-bst/mhw2_task1.txt
```

# Задание 2.1

Найти топ 10 самых популярных фильтров. Фильтры кодируются ключами в словаре SearchParams (именно ключи (числа), а не значения).

```bash
mkdir mhw2
kaggle competitions download -c avito-context-ad-clicks -f SearchInfo.tsv.7z
kaggle competitions download -c avito-context-ad-clicks -f trainSearchStream.tsv.7z
kaggle competitions download -c avito-context-ad-clicks -f testSearchStream.tsv.7z
kaggle competitions download -c avito-context-ad-clicks -f AdsInfo.tsv.7z

mv SearchInfo.tsv.7z mhw2
mv trainSearchStream.tsv.7z mhw2
mv testSearchStream.tsv.7z mhw2
mv AdsInfo.tsv.7z mhw2
cd mhw2

7z x SearchInfo.tsv.7z 
7z x trainSearchStream.tsv.7z
7z x testSearchStream.tsv.7z
7z x AdsInfo.tsv.7z
```